In [1]:
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [2]:
from itertools import islice
import time

In [3]:
from Surfer import GetBrowser,RedditFeed,Feed,ChatGPT,ChatGPTWebsite
from Surfer.ScriptRunner import ScriptRunner
from files import *
from Stegasus import stegasus
from Bot import *
from Chatter import prompts
from Surfer.util import uuid

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nasoo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Setting up Masked Stego



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Setting up Masked Stego Completed


Setting up Typoceros

('hi, how atre you?', '01001011111')

Setting up Typoceros Completed



In [4]:
# Example usage
ex = "svg"
file_path = f'tiny.{ex}'
data = ''
for chunk in read_file_in_chunks(file_path):
    data += chunk
data

'001111000111001101110110011001110010000001111000011011010110110001101110011100110011110100100010011010000111010001110100011100000011101000101111001011110111011101110111011101110010111001110111001100110010111001101111011100100110011100101111001100100011000000110000001100000010111101110011011101100110011100100010001000000110011001101001011011000110110000111101001000100110111001101111011011100110010100100010001000000111011001101001011001010111011101000010011011110111100000111101001000100011000000100000001100000010000000110010001101000010000000110010001101000010001000100000011100110111010001110010011011110110101101100101001011010111011101101001011001000111010001101000001111010010001000110001001011100011010100100010000011010000101000001001011100110111010001110010011011110110101101100101001111010010001001100011011101010111001001110010011001010110111001110100010000110110111101101100011011110111001000100010001000000110000101110010011010010110000100101101011010000110100101100100011001000110010

In [57]:
# KB
data = data[:2**10]

In [5]:
import random

In [6]:
cache = {}

In [8]:
class Encoder:
	def __init__(self,data,dataLog,encodedChatLog):
		self.data = data
		self.rem = data
		self.dataLog = dataLog
		self.encodedChatLog = encodedChatLog
		self.dataLog.append('')
		self.sending_pure_padding = False

	def done(self):
		return self.sending_pure_padding

	def __call__(self,text) -> str:
		# sending pure padding
		self.sending_pure_padding = len(self.rem) < 499
		a, self.rem = stegasus.StegasusEncode(text,self.rem)
		self.encodedChatLog.append(a)
		self.dataLog.append(data[len(self.dataLog[-1]):-len(self.rem)])
		return a


In [10]:
from typing import List


def openChannelAndSend(data:str
, alice = Person('alice','nice',23,'female','Boston','alice@gmail.com')
, bob = Person('bob','wick',29,'male','Boston','bob@gmail.com')):
	runLog = []
	cache[uuid()] = runLog
	data += '1' + ('0' * 500)
	cache['data'] = data
	
	driver = cache.get('driver',GetBrowser())
	driver.implicitly_wait(2)
	rf = cache.get('RedditFeed',RedditFeed(driver,skip=False,subreddit='unpopularopinion',newWindow=False))
	# rf = RedditFeed(driver,skip=True)
	posts = []
	if 'posts' in cache:
		posts = cache['posts']
	else:
		for x in islice(rf,5):
			posts.append(x)
			time.sleep(2*max(1,len(x) * 0.01))
		random.shuffle(posts)
		cache['posts'] = posts

	i = 0
	while i < len(posts) \
		and posts[i]['excerpt'] is not None \
		and posts[i]['excerpt'] != "" \
		and posts[i]['title'] is not None \
		and posts[i]['title'] != "":
		i+=1
		
	gptWebsite = cache.get('ChatGPTWebsite',ChatGPTWebsite(driver,skip=False))
	
	time.sleep(7)
	
	encodedChatLog: List[str] = []
	cache['encodedChatLog'] = encodedChatLog
	chatLog: List[Message] = []
	cache['chatLog'] = chatLog
	promptLog: List[str] = []
	cache['promptLog'] = promptLog
	dataLog: List[str] = []
	cache['dataLog'] = dataLog
	
	encoder = Encoder(data,dataLog,encodedChatLog)

	aliceChat = cache.get('aliceChat', gptWebsite.getLandingNewChat())
	if len(promptLog) == 0:
		# promptLog 0
		promptLog.append(prompts.start_conversation_with_post(alice,bob,posts[i]))
	
	if len(chatLog) == 0:
		# chatLog 0
		chatLog.append(Message(alice,aliceChat.ask(promptLog[0])))
	
	encoder(chatLog[0].text)

	if 'bobChat' not in cache or len(chatLog) == 1 or len(promptLog) == 1:
		# promptLog 1
		promptLog.append(prompts.startChatReplyPrompt(chatLog[0]))
		a,bobChat = aliceChat.askNew(promptLog[1])
		# chatLog 1
		chatLog.append(Message(bob,a))
		cache['bobChat'] = bobChat
	else:
		bobChat =cache['bobChat']

	# chatLog 2
	chatLog.append(Message(alice, aliceChat.ask(prompts.startChatReplyPrompt(chatLog[1]))))
	a = encoder(chatLog[2].text)
	
	cIdx = 2
	while True:
		if encoder.done():
			# chatLog cIdx+1
			chatLog.append(Message(alice, aliceChat.ask(prompts.endChatReplyPrompt(chatLog[cIdx]))))
			cIdx += 1
			a = encoder(chatLog[cIdx].text)
			return None

		# chatLog cIdx+1
		chatLog.append(Message(bob, bobChat.ask(prompts.inChatReplyPrompt(chatLog[cIdx]))))
		cIdx += 1
		
		# switch
		
		# chatLog cIdx+1
		chatLog.append(Message(alice, aliceChat.ask(prompts.inChatReplyPrompt(chatLog[cIdx]))))
		cIdx += 1
		a = encoder(chatLog[cIdx].text)


In [11]:
openChannelAndSend(data)

In [12]:
len(data)

2128

In [13]:
cache

{'axLQKuFYincYCwchKxFB': [],
 'data': '0011110001110011011101100110011100100000011110000110110101101100011011100111001100111101001000100110100001110100011101000111000000111010001011110010111101110111011101110111011100101110011101110011001100101110011011110111001001100111001011110011001000110000001100000011000000101111011100110111011001100111001000100010000001100110011010010110110001101100001111010010001001101110011011110110111001100101001000100010000001110110011010010110010101110111010000100110111101111000001111010010001000110000001000000011000000100000001100100011010000100000001100100011010000100010001000000111001101110100011100100110111101101011011001010010110101110111011010010110010001110100011010000011110100100010001100010010111000110101001000100000110100001010000010010111001101110100011100100110111101101011011001010011110100100010011000110111010101110010011100100110010101101110011101000100001101101111011011000110111101110010001000100010000001100001011100100110100101100001001011010

In [17]:
import pandas as pd
import math

In [52]:
df = pd.read_csv('./stats.tsv',delimiter='\t',index_col=0)

In [53]:
df.to_csv('./stats.tsv',sep='\t',index=False)

In [54]:
df.describe()

,bits,cover size,capacity
count,57.000000,57.000000,57.000000
mean,86.631579,187.964912,46.385965
std,22.304413,42.138786,7.143315
min,33.000000,85.000000,27.000000
25%,74.000000,165.000000,42.000000
50%,88.000000,202.000000,46.000000
75%,95.000000,215.000000,52.000000
max,166.000000,318.000000,61.000000


In [33]:
# df.drop('in',axis=1,inplace=True)
# df.drop('encoded',axis=1,inplace=True)

# df

In [55]:
dfd = df.describe()
dfd['bits'] = dfd['bits'].apply(lambda a: int(round(a)))
dfd['cover size'] = dfd['cover size'].apply(lambda a: int(round(a)))
dfd['capacity'] = dfd['capacity'].apply(lambda a: int(round(a)))
dfd

,bits,cover size,capacity
count,57,57,57
mean,87,188,46
std,22,42,7
min,33,85,27
25%,74,165,42
50%,88,202,46
75%,95,215,52
max,166,318,61


In [56]:
dfd.to_csv('./stats.stats.tsv',sep='\t')